# Introduction and Pre-Processing

In this notebook, we explore a handful of the preprocessing tools BART offers.

### Setup

If you are runnning locally, you will need [BART](https://www.github.com/mrirecon/bart) as well as [View](https://www.github.com/mrirecon/view), an open-source tool to view data. Once both are installed, run the following cell to add the necessary directories to your `PATH`. Make sure to change the placeholder directories to the directories where you have installed BART and View.

In [1]:
export PATH=/path/to/bart:$PATH
export PATH=/path/to/view:$PATH

### File Formats and Metadata

As a motivating example, we will be reconstructing a slice of a 3D FSE scan of a knee. The data is available [here](http://old.mridata.org/fullysampled/knees) if you are looking to recreate these results on your own, or experiment on other MR datasets. The following cell will download and unpack the data for ease of use and navigation

In [ ]:
wget http://old.mridata.org/knees/fully_sampled/p1/e1/s1/P1.zip
unzip P1.zip
mv p1/e1/s1/* .
rmdir p1

Before viewing the image, we are going to use `bart show` to get a better understanding of the data we are working with. BART data is stored in two files, a header (`.hdr`) file which contains metadata (dimensions, file type, etc) and a complex float (`.cfl`) file which contains the raw data. By default, `show` will print the contents of the `.cfl` file. To view the metadata, append the `-m` flag to the command.

In [23]:
bart show -m kspace

dbus[2079]: Dynamic session lookup supported but failed: launchd did not provide a socket path, verify that org.freedesktop.dbus-session.plist is loaded!
Type: complex float
Dimensions: 16
AoD:	320	320	256	8	1	1	1	1	1	1	1	1	1	1	1	1


The `AoD` header tells us that we are working with k-space data of 320 values in the spatial frequency/$k_x$ direction, and 320 values in the phase encoding/$k_y$ direction. We have 256 slices, and the data has been collected from 8 coils. Let's take a look in `view`.

In [24]:
view kspace&

[4] 2084


: 1

**TODO**: What do BART sliders parameterize? Indicate here how users can recreate results, but explain what settings are doing. Additionally, would showing how `view` lets a user view the phase be of interest here as well?

**SETTINGS**
First slider: 0
Second: 0.001
Third 1.4
Fourth 1.0

We are going to extract the slice 128. Using the column on the left (for those following along: click the settings icon) we can choose which slice to view. We can adjust the value below this one to choose the coil to view the results from. 

### Performing Inverse FFT Along the Readout Direction

**TODO**: Motivation for why we do this

Prior to extracting a slice, we perform an inverse FFT along the readout direction

Now, we are going to demonstrate how to perform a simple inverse FFT on the k-space data. Note: to specify the dimensions BART operates on, one must pass in an appropriate bitmask: (e.g., for dimensions 0 and 1, this bitmask is calculated as $2^0 + 2^1 = 3$). `bart bitmask` automatically computes this value for us.

In [14]:
bart fft -h

Usage: fft [-u] [-i] [-n] bitmask <input> <output>

Performs a fast Fourier transform (FFT) along selected dimensions.

-u		unitary
-i		inverse
-n		un-centered
-h		help


`-u` and `-i` indicate that we want a unitary and inverse FFT respectively.

In [10]:
bart fft -u -i $(bart bitmask 2) kspace kspace_fft

[1]+  Done                    view knee_fft


Let's examine the dimensions of the result of applying the inverse FFT, and take a look in view. 

In [11]:
bart show -m kspace_fft

Type: complex float
Dimensions: 16
AoD:	320	320	256	8	1	1	1	1	1	1	1	1	1	1	1	1


In [13]:
view kspace_fft&

[2] 1711


: 1

### Extracting a Slice Along the Readout Direction

`bart slice` allows us to extract a designated slice of an image along a specified dimension. We will extract a slice along the readout direction to later perform an FFT on. 

In [5]:
bart slice -h

dbus[4186]: Dynamic session lookup supported but failed: launchd did not provide a socket path, verify that org.freedesktop.dbus-session.plist is loaded!
Usage: slice dim1 pos1 ... dimn posn <input> <output>

Extracts a slice from positions along dimensions.


-h		help


Recalling our image metadata, we can see that dimension `2` corresponds to the slices of the 3D image. We are going to extract the slice at position `128`, so our command looks as follows:

In [6]:
bart slice 2 128 kspace_fft ksp_slice

Using `show` we verify that our slice dimensions look correct.

In [7]:
bart show -m ksp_slice

Type: complex float
Dimensions: 16
AoD:	320	320	1	8	1	1	1	1	1	1	1	1	1	1	1	1


Dimensions look good! We have preserved the $k_x$ and $k_y$ dimensions as well as the number of coils. Let's look at the image in `view`

In [8]:
view ksp_slice&

[2] 2035


: 1

### Resizing

`bart resize` can be used to manipulate data. We'll briefly explore resizing our slice with the tool.

In [2]:
bart resize -h

Usage: resize [-c] dim1 size1 ... dimn sizen <input> <output>

Resizes an array along dimensions to sizes by truncating or zero-padding.

-c		center
-h		help


In [3]:
bart resize -c 0 160 1 160 ksp_slice resized_slice

In [4]:
view resized_slice&

[1] 4341


We've adjusted the dime

### Coil Compression

We will now use BART's build in coil compression method, `cc`, to compress to four virtual coils. The `-G` flag enables [Geometric](https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.24267) coil compression, but additional flags offer SVD and ESPIRiT based methods as well.

In [10]:
bart cc -p 4 -G ksp_slice ksp_cc

Done.


`-p` allows us to specify the number of virtual coils we wish to compress to.

In [11]:
bart show -m ksp_cc

Type: complex float
Dimensions: 16
AoD:	320	320	1	4	1	1	1	1	1	1	1	1	1	1	1	1


Dimension `3` - the number of coils - has changed to reflect our compression. This looks good, let's take a look at the image in `view`.

In [12]:
view ksp_cc&

[3] 2159


Note that in now we can only toggle the four virtual coils on the right-hand column.

### Reconstruction via FFT

Now, we're going to apply a basic inverse FFT to the K-Space slice to reconstruct the Knee image. Once again, we specify a bitmask, but this time we want to apply our FFT to the dimensions `0` and `1`, or reconstruct our image signal from the spatial frequency and phase encoding dimensions, respectively.

In [14]:
bart fft -i -u $(bart bitmask 0 1) ksp_cc knee

[1]-  Done                    view knee_fft
[2]+  Done                    view kspace_fft


In [15]:
view knee

dbus[1721]: Dynamic session lookup supported but failed: launchd did not provide a socket path, verify that org.freedesktop.dbus-session.plist is loaded!


Note that we're not done quite yet. If we cycle through the virtual coils in View (adjusting the value of the fourth dimension in the toolbar on the right) we can see that each coil is still contributing different sensitivities to the final image. Let's combine the coils using a Root-Sum-of-Squares (`rss`) calculation as follows:

In [21]:
bart rss $(bart bitmask 3) knee knee_combined

In [22]:
view knee_combined&

[3] 2079
